In [76]:
# https://www.tensorflow.org/decision_forests/tutorials/beginner_colab
# https://www.tensorflow.org/decision_forests/tutorials/predict_colab
# https://www.tensorflow.org/decision_forests/tutorials/automatic_tuning_colab
# https://www.tensorflow.org/decision_forests/tutorials/dtreeviz_colab
from google.cloud import aiplatform
from google.cloud import bigquery

import pandas as pd
import numpy as np

import math
import os

import tensorflow as tf
print(tf.__version__)

2.12.0


In [77]:
# ! pip install --upgrade tensorflow-decision-forests
import tensorflow_decision_forests as tfdf
print(tfdf.__version__)

1.3.0


In [102]:
#! pip install -q -U dtreeviz
import dtreeviz
print(dtreeviz.__version__)

2.2.1


# Variable to Configuration

In [78]:
projectId='pongthorn'
dataset_id='SMartML'

train_name='train2_incident'
test_name='test2_incident'

train_table_id=f"{projectId}.{dataset_id}.{train_name}"
test_tabel_id=f"{projectId}.{dataset_id}.{test_name}"
print(f"train-ds = {train_table_id}")
print(f"test-ds = {test_tabel_id}")

train-ds = pongthorn.SMartML.train2_incident
test-ds = pongthorn.SMartML.test2_incident


In [79]:
metric="accuracy"
model_gs_path="gs://demo-tuned-tf-incident-pongthorn/model_df_tf"

In [80]:
option=1
if option==1:

    unusedCols=['id','severity_id','severity_name','label_binary_severity','open_to_close_hour','response_to_resolved_hour']
    cateCols=['sla','product_type','brand','service_type','incident_type','range_open_to_close_hour','range_response_to_resolved_hour']
    numbericCols=[]
else:
    unusedCols=['id','severity_id','severity_name','label_binary_severity','range_open_to_close_hour','range_response_to_resolved_hour']
    cateCols=['sla','product_type','brand','service_type','incident_type']
    numbericCols=['open_to_close_hour','response_to_resolved_hour']


labelCol='label_multi_severity'
# labelCol='label_binary_severity'

# Load & Prepare Data

In [81]:
def load_ml_data(data_path):
 df=pd.read_csv(data_path)
 df =df.drop(columns=unusedCols)
 
 return df

def load_data_bq(sql:str):
 
 query_result=client.query(sql)
 df=query_result.to_dataframe()
 df =df.drop(columns=unusedCols)
 df[labelCol]=df[labelCol].astype('int64') 
 df=df[[labelCol]+cateCols+numbericCols]   
  
 return df

In [82]:
client = bigquery.Client(project=projectId)

train=load_data_bq(f"SELECT * FROM {train_table_id}")
test=load_data_bq(f"SELECT * FROM {test_tabel_id}")

print(train.info())

print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2104 entries, 0 to 2103
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   label_multi_severity             2104 non-null   int64 
 1   sla                              2104 non-null   object
 2   product_type                     2104 non-null   object
 3   brand                            2104 non-null   object
 4   service_type                     2104 non-null   object
 5   incident_type                    2104 non-null   object
 6   range_open_to_close_hour         2104 non-null   object
 7   range_response_to_resolved_hour  2104 non-null   object
dtypes: int64(1), object(7)
memory usage: 131.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527 entries, 0 to 526
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 

In [83]:
train.tail(5)

,label_multi_severity,sla,product_type,brand,service_type,incident_type,range_open_to_close_hour,range_response_to_resolved_hour
2099,2,24x7 4Hrs Response Time,Software,Trend Micro,Incident,General Incident,worst,worst
2100,2,24x7 6Hrs Resolution Time,Software,Trend Micro,Incident,General Incident,best,best
2101,2,24x7 4Hrs Response Time,Security,Trend Micro,Incident,Software,good,good
2102,2,24x7 6Hrs Resolution Time,Software,Trend Micro,Incident,Configuration Change,good,good
2103,2,24x7 6Hrs Resolution Time,Software,Trend Micro,Incident,General Incident,moderate,moderate


In [84]:
test.tail()

,label_multi_severity,sla,product_type,brand,service_type,incident_type,range_open_to_close_hour,range_response_to_resolved_hour
522,2,24x7 4Hrs Resolution Time,Firewall,Palo Alto,Incident,General Incident,best,best
523,1,24x7 4Hrs Resolution Time,Firewall,Palo Alto,Incident,General Incident,good,good
524,0,24x7 4Hrs Response Time,Firewall,Palo Alto,Request,Configuration Change,bad,bad
525,1,24x7 4Hrs Response Time,Software,Trend Micro,Incident,Software,best,best
526,0,24x7 4Hrs Response Time,Software,Trend Micro,Request,General Incident,moderate,good


# Build Model

In [85]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label=labelCol)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, label=labelCol)

In [86]:
# Specify the model.
#model = tfdf.keras.RandomForestModel()
model=tfdf.keras.GradientBoostedTreesModel()
model.fit(x=train_ds)

Use /var/tmp/tmp9soe6f3_ as temporary training directory
Reading training dataset...


[WARNING 23-07-01 18:13:10.9326 UTC gradient_boosted_trees.cc:1797] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-07-01 18:13:10.9337 UTC gradient_boosted_trees.cc:1808] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-07-01 18:13:10.9342 UTC gradient_boosted_trees.cc:1822] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
2023-07-01 18:13:10.957337: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype string and shape [2104]
	 [[{{node Placeholder/_4}}]]


Training dataset read in 0:00:00.237900. Found 2104 examples.
Training model...
Model trained in 0:00:00.163307
Compiling model...


[INFO 23-07-01 18:13:11.3343 UTC kernel.cc:1242] Loading model from path /var/tmp/tmp9soe6f3_/model/ with prefix aa574ca90f0241f8
[INFO 23-07-01 18:13:11.3472 UTC decision_forest.cc:660] Model loaded with 76 root(s), 3708 node(s), and 7 input feature(s).
[INFO 23-07-01 18:13:11.3481 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 23-07-01 18:13:11.3486 UTC kernel.cc:1074] Use fast generic engine
2023-07-01 18:13:11.369624: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [2104]
	 [[{{node Placeholder/_2}}]]


Model compiled.


In [87]:
model.compile(metrics=[metric])
evaluation = model.evaluate(test_ds, return_dict=True)
for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

2023-07-01 18:13:11.835750: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int64 and shape [527]
	 [[{{node Placeholder/_7}}]]


1/1 [==============================] - 0s 165ms/step - loss: 0.0000e+00 - accuracy: 0.7324
loss: 0.0000
accuracy: 0.7324


# Save Model

In [88]:
model.save(model_gs_path)

INFO:tensorflow:Assets written to: gs://demo-tuned-tf-incident-pongthorn/model_df_tf/assets


INFO:tensorflow:Assets written to: gs://demo-tuned-tf-incident-pongthorn/model_df_tf/assets


# Visualization Model

In [104]:
import logging
from IPython import display
logging.getLogger('matplotlib.font_manager').setLevel(level=logging.CRITICAL)

display.set_matplotlib_formats('retina') # generate hires plots

/var/tmp/ipykernel_3311/1867066456.py:5: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`


In [ ]:
# https://www.tensorflow.org/decision_forests/tutorials/dtreeviz_colab